In [ ]:
# !pip install pdfminer.six
# !pip install spacy 
# !python3 -m spacy download en_core_web_sm

In [37]:
import spacy,re,glob
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams

# 设置extract_text()的参数，未设置部分单词会存在没有空格的情况
laparams = LAParams()
setattr(laparams, 'all_texts', True)
# 导入data目录下的所有pdf
pdfs = glob.glob("data/*.pdf")
text = ""
for pdf in pdfs:
    temp = extract_text(pdf, laparams=laparams)
    # 清除多余的 符号
    temp = re.sub(r"’ll|n’t|’s|’re|[^a-zA-Z]+",' ',temp.lower())
    text += temp


# 通过Spacy 还原单词原型，并统计单词的出现频次
nlp = spacy.load("en_core_web_sm")
# 默认的nlp处理的文本大小1000000,报错的话 把max_length设置大一点
nlp.max_length = 2000000
doc = nlp(text)
dicts = {}
for token in doc:
    if len(token.lemma_) > 2:
        dicts[token.lemma_] = dicts.get(token.lemma_,0)+1

# 按照词出现频次按照降序排序
# ls = list(dicts.items())
# ls.sort(key = lambda x:x[1], reverse=True)
# print(dicts)



In [34]:
import pandas as pd
# 转置，导入pandas 进行分析
df=pd.DataFrame([dicts]).T
df=df.reset_index().rename(columns={'index':'word',0:'count'})
t = df[(df["word"]!="burningvocabulary") & (df["count"]>=2)].sort_values(by="count",ascending=False)
t

,word,count
10,the,8279
14,be,4956
2,of,4422
35,to,4275
18,and,3039
...,...,...
5497,violent,2
946,simplicity,2
5495,unconventional,2
5491,misinterpreted,2


In [36]:
# 这部分 减去高考单词，然后再统计
gaokao = pd.read_csv("data/gaokaowords.csv",sep=' ',usecols=[0],names=['word'])
# 使单词转成小写
gaokao["word"] = gaokao["word"].str.lower()
gaokao["isgaokao"]=1

data = pd.merge(df, gaokao, on="word", how="left")

dd = data[(data["isgaokao"]!=1)&(data["count"]>=2)].sort_values(by="count",ascending=False)
dd

,word,count,isgaokao
301,more,617,NaN
138,an,518,NaN
153,com,412,NaN
152,burningvocabulary,407,NaN
864,should,365,NaN
...,...,...,...
5449,exclusively,2,NaN
5448,seditionact,2,NaN
5446,scalia,2,NaN
5445,antonin,2,NaN
